In [1]:
import pandas as pd
csv_file = "/Users/i337036/Downloads/train.csv"
df = pd.read_csv(csv_file, encoding='utf-8')
len(df)

404290

In [2]:
df.columns.values

array(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'],
      dtype=object)

In [3]:
df = df[df['is_duplicate']==1]

In [4]:
df = df.drop_duplicates(subset=['question1'], keep='first')
df = df.drop_duplicates(subset=['question2'], keep='first')
len(df)

73776

In [5]:
q1_list = df.question1.tolist()
q2_list = df.question2.tolist()
print(len(q1_list), len(q2_list))

73776 73776


In [6]:
q1q2_list = ["%s %s" % (txt1, txt2) for txt1, txt2 in zip(q1_list, q2_list)]
print(len(q1q2_list))

73776


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', min_df = 0, stop_words = 'english')
tf_mat = tf.fit(q1q2_list)

In [8]:
q2_mat = tf.transform(q2_list)
q1_mat = tf.transform(q1_list)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

def find_similar(q1_mat, q2_mat, index, top_n=5):
    qd_sims = cosine_similarity(q1_mat[index: index+1], q2_mat).flatten()
    qd_indices = qd_sims.argsort()[:-top_n:-1]
    return qd_indices

search_idx = 10
print("Searched for %s at [%d]" % (q1_list[search_idx], search_idx))
for index in find_similar(q1_mat, q2_mat, search_idx):
    print ("%s %d" % (q2_list[index], index))

Searched for What are some special cares for someone with a nose that gets stuffy during the night? at [10]
How can I keep my nose from getting stuffy at night? 10
What should I do if my kitten has a stuffy nose? 15492
What are some special cares when you have a dry throat and a runny nose? 3998
How can you see if someone cares about you? 62798


In [10]:
q1_list[search_idx], q2_list[search_idx]

('What are some special cares for someone with a nose that gets stuffy during the night?',
 'How can I keep my nose from getting stuffy at night?')

In [11]:
from tqdm import tqdm

In [13]:
correct_index = 0
for search_idx in tqdm(range(500)):
    for index in find_similar(q1_mat, q2_mat, search_idx):
        if search_idx==index or q1_list[search_idx]==q2_list[index]:
            correct_index += 1
            break

100%|██████████| 500/500 [00:03<00:00, 149.03it/s]


In [14]:
print(correct_index)

367
